In [1]:
import sqlite3
import pandas as pd

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('courses.db')
cursor = conn.cursor()

# Function to load CSV data into SQLite tables
def load_csv_to_sqlite(csv_file, table_name, conn):
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Load CSV files into SQLite
load_csv_to_sqlite('Courses.csv', 'Courses', conn)
load_csv_to_sqlite('Learning_Outcomes.csv', 'Learning_Outcomes', conn)
load_csv_to_sqlite('Instructors.csv', 'Instructor', conn)
load_csv_to_sqlite('Course_Assignment.csv', 'Course_Assignment', conn)

In [2]:
# Define and execute the query
myquery = '''
SELECT mnemonic, course_name
FROM Courses
WHERE active = 1
'''

# Fetch and print results
active_courses = pd.read_sql_query(myquery, conn)
print("Active Courses:")
print(active_courses, "\n")

Active Courses:
            mnemonic                               course_name
0    ds_biz_anaytics    Business Analytics for Data Scientists
1   ds_tech_bootcamp                        Technical Bootcamp
2             ds5001                Exploratory Text Analytics
3             ds5012           Foundations of Computer Science
4             ds5100              Programming for Data Science
5             ds5110                          Big Data Systems
6             ds6001  Practice and Application of Data Science
7             ds6002                           Big Data Ethics
8             ds6011      Data Science Capstone Project Work I
9             ds6013     Data Science Capstone Project Work II
10            ds6030                      Statistical Learning
11            ds6040                 Bayesian Machine Learning
12            ds6050                             Deep Learning
13          sarc5400                        Data Visualization
14          stat6021            Linear 

In [3]:
# Define and execute the query
myquery = '''
SELECT mnemonic, course_name
FROM Courses
WHERE active = 0
'''

# Fetch and print results
inactive_courses = pd.read_sql_query(myquery, conn)
print("Inactive Courses:")
print(inactive_courses, "\n")


Inactive Courses:
  mnemonic                                      course_name
0   ds6003  Practice and Application of Data Science Part 2
1   ds6012                           Big Data Ethics Part 2 



In [4]:
# Define and execute the query
myquery = '''
SELECT instructor_name
FROM Instructor
WHERE active = 0
'''

# Fetch and print results
inactive_instructors = pd.read_sql_query(myquery, conn)
print("Inactive Instructors:")
print(inactive_instructors, "\n")

Inactive Instructors:
              instructor_name
0               Jeremy Bolton
1  Luis Felipe Rosado Murillo 



In [5]:
myquery = '''
SELECT c.mnemonic, c.course_name, COUNT(l.outcome_id) AS number_of_learning_outcomes
FROM Courses c
LEFT JOIN Learning_Outcomes l ON c.course_id = l.course_id
GROUP BY c.mnemonic, c.course_name
'''

# Fetch and print results
learning_outcomes_count = pd.read_sql_query(myquery, conn)
print("Learning Outcomes Count:")
print(learning_outcomes_count, "\n")

Learning Outcomes Count:
            mnemonic                                      course_name  \
0             ds5001                       Exploratory Text Analytics   
1             ds5012                  Foundations of Computer Science   
2             ds5100                     Programming for Data Science   
3             ds5110                                 Big Data Systems   
4             ds6001         Practice and Application of Data Science   
5             ds6002                                  Big Data Ethics   
6             ds6003  Practice and Application of Data Science Part 2   
7             ds6011             Data Science Capstone Project Work I   
8             ds6012                           Big Data Ethics Part 2   
9             ds6013            Data Science Capstone Project Work II   
10            ds6030                             Statistical Learning   
11            ds6040                        Bayesian Machine Learning   
12            ds6050      

In [6]:
# Define and execute the query
myquery = '''
SELECT c.mnemonic, c.course_name
FROM Courses c
LEFT JOIN Learning_Outcomes l ON c.course_id = l.course_id
WHERE l.outcome_id IS NULL
'''

# Fetch and print results
courses_with_no_outcomes = pd.read_sql_query(myquery, conn)
print("Courses with No Learning Outcomes:")
print(courses_with_no_outcomes, "\n")

Courses with No Learning Outcomes:
   mnemonic         course_name
0  sarc5400  Data Visualization 



In [7]:
# Define and execute the query
myquery = '''
SELECT c.mnemonic, c.course_name, l.learning_outcome
FROM Courses c
JOIN Learning_Outcomes l ON c.course_id = l.course_id
WHERE l.learning_outcome LIKE '%SQL%'
'''

# Fetch and print results
courses_with_SQL = pd.read_sql_query(myquery, conn)
print("Courses with SQL Learning Outcomes:")
print(courses_with_SQL, "\n")

Courses with SQL Learning Outcomes:
  mnemonic                               course_name  \
0   ds5110                          Big Data Systems   
1   ds6001  Practice and Application of Data Science   
2   ds6001  Practice and Application of Data Science   
3   ds6001  Practice and Application of Data Science   

                                    learning_outcome  
0             Apply Spark SQL to data analysis tasks  
1  Understand the purpose, typology, and language...  
2  Understand how to implement databases Python: ...  
3        Understand how to query databases with SQL    



In [8]:
myquery = '''
SELECT ca.course_id, ca.term, i.instructor_name
FROM Course_Assignment ca
JOIN Instructor i ON ca.instructor_id = i.instructor_id
WHERE ca.course_id = 104
  AND ca.term = 'summer2021'
'''
# Fetch and print results
assignments_info = pd.read_sql_query(myquery, conn)
print("Course Assignments for 'ds5100' in Summer 2021:")
print(assignments_info, "\n")

Course Assignments for 'ds5100' in Summer 2021:
   course_id        term instructor_name
0        104  summer2021      Nada Basit 



In [12]:
myquery = '''
SELECT DISTINCT i.instructor_name
FROM Course_Assignment ca
JOIN Instructor i ON ca.instructor_id = i.instructor_id
WHERE ca.term = 'fall2021'
ORDER BY i.instructor_name
'''

# Fetch and print results
instructors_fall_2021 = pd.read_sql_query(myquery, conn)
print("Instructors who taught in Fall 2021:")
print(instructors_fall_2021, "\n")


Instructors who taught in Fall 2021:
     instructor_name
0   Abbas Kazemipour
1       Adam Tashman
2       Bill Basener
3     Cait Dreisbach
4       Eric Tassone
5        Jeffrey Woo
6      Jeremy Bolton
7           Judy Fox
8            Lei Xie
9      Marc Ruggiano
10       Pete Alonzi
11      Peter Gedeck
12      Raf Alvarado
13    Renee Cummings
14  Sree Mallikarjun
15      Stephen Baek
16      Teague Henry 



In [10]:
# Define and execute the query
myquery = '''
WITH InstructorCount AS (
    SELECT course_id, term, COUNT(DISTINCT instructor_id) AS instructor_count
    FROM Course_Assignment
    GROUP BY course_id, term
)
SELECT c.mnemonic, ca.term
FROM InstructorCount ic
JOIN Course_Assignment ca ON ic.course_id = ca.course_id AND ic.term = ca.term
JOIN Courses c ON ca.course_id = c.course_id
WHERE ic.instructor_count > 1
GROUP BY c.mnemonic, ca.term
'''

# Fetch and print results
courses_multiple_instructors = pd.read_sql_query(myquery, conn)
print("Courses with Multiple Instructors:")
print(courses_multiple_instructors, "\n")


Courses with Multiple Instructors:
  mnemonic        term
0   ds6011    fall2021
1   ds6011  summer2021
2   ds6013    fall2021 



In [11]:

myquery = '''
WITH InstructorCount AS (
    SELECT course_id, term, COUNT(DISTINCT instructor_id) AS instructor_count
    FROM Course_Assignment
    GROUP BY course_id, term
)
SELECT ca.term, c.mnemonic, i.instructor_name
FROM InstructorCount ic
JOIN Course_Assignment ca ON ic.course_id = ca.course_id AND ic.term = ca.term
JOIN Courses c ON ca.course_id = c.course_id
JOIN Instructor i ON ca.instructor_id = i.instructor_id
WHERE ic.instructor_count > 1
ORDER BY ca.term, c.mnemonic, i.instructor_name
'''

# Fetch and print results
courses_multiple_instructors_details = pd.read_sql_query(myquery, conn)
print("Details for Courses with Multiple Instructors:")
print(courses_multiple_instructors_details, "\n")

Details for Courses with Multiple Instructors:
         term mnemonic   instructor_name
0    fall2021   ds6011  Abbas Kazemipour
1    fall2021   ds6011    Cait Dreisbach
2    fall2021   ds6011      Eric Tassone
3    fall2021   ds6011     Jeremy Bolton
4    fall2021   ds6011           Lei Xie
5    fall2021   ds6011      Peter Gedeck
6    fall2021   ds6013      Raf Alvarado
7    fall2021   ds6013      Teague Henry
8  summer2021   ds6011      Raf Alvarado
9  summer2021   ds6011      Teague Henry 

